In [1]:
# setting up all the packages
# setting up all the packages
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import torch
import torch.nn as nn

import rasterio
from rasterio.enums import Resampling

#from utils import * 

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

plt.rcParams['savefig.dpi'] = 400
plt.rcParams['font.size'] = 13
plt.rcParams["legend.frameon"] = False

In [2]:
# establish working directory
print(os.getcwd())

/home/jovyan/MLEAEEE4000-DroughtAmazon2


In [3]:
# read in the wtd tiff file using rasterio
with rasterio.open("/home/jovyan/MLEAEEE4000-DroughtAmazon2/SAMERICA_WTD_annualmean.tiff") as src:
    data = src.read(1)        # read first band
    profile = src.profile     # metadata
    print(data.shape)
    print(profile)


(8520, 7320)
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.3999999521443642e+38, 'width': 7320, 'height': 8520, 'count': 1, 'crs': CRS.from_wkt('GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]'), 'transform': Affine(0.008333300254207799, 0.0, -93.00000100071304,
       0.0, -0.008333300212023194, 14.99971680574566), 'blockxsize': 7320, 'blockysize': 1, 'tiled': False, 'compress': 'lzw', 'interleave': 'band'}


In [4]:
# get crs and scale of wtd tiff
with rasterio.open("/home/jovyan/MLEAEEE4000-DroughtAmazon2/SAMERICA_WTD_annualmean.tiff") as src:
    print(src.crs)   # EPSG:4326 means in degrees
    print(src.res)   # resolution in degrees

EPSG:4326
(0.008333300254207799, 0.008333300212023194)


In [6]:
# get crs and scale of the mdd extent
with rasterio.open("/home/jovyan/MLEAEEE4000-DroughtAmazon2/mdd_extent_TCscale.tif") as src:
    print(src.crs)   # EPSG:4326 means in degrees
    print(src.res) 

EPSG:4326
(0.04166666666666666, 0.04166666666666666)


In [10]:
# import the transform functions from rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

In [11]:
raster_A = "/home/jovyan/MLEAEEE4000-DroughtAmazon2/SAMERICA_WTD_annualmean.tiff"   # input raster (degrees)
raster_B = "/home/jovyan/MLEAEEE4000-DroughtAmazon2/mdd_extent_TCscale.tif" # raster whose resolution you want to match
output = "/home/jovyan/MLEAEEE4000-DroughtAmazon2/SAMERICA_WTD_rescaled.tif"

# Open raster B to read the desired resolution and bounds ---
with rasterio.open(raster_B) as ref:
    dst_crs = ref.crs
    dst_res = ref.res
    dst_bounds = ref.bounds

# Open raster A for resampling ---
with rasterio.open(raster_A) as src:

    # Compute the transform + output size for the new resolution
    transform, width, height = calculate_default_transform(
        src.crs,
        dst_crs,
        src.width,
        src.height,
        *src.bounds,
        resolution=dst_res  # << key step
    )

    # Prepare metadata
    kwargs = src.meta.copy()
    kwargs.update({
        "crs": dst_crs,
        "transform": transform,
        "width": width,
        "height": height
    })

    # Write output
    with rasterio.open(output, "w", **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.average,  # good for downscaling (coarser)
            )

In [ ]:
# clip the reprojected raster to MDD political boundaries

In [12]:
import geopandas as gpd

In [13]:
mdd = gpd.read_file("/home/jovyan/MLEAEEE4000-DroughtAmazon2/output.geojson")
mdd = mdd.to_crs("EPSG:4326")   # set crs to the crs of the raster

In [14]:
from rasterio.mask import mask

In [15]:
raster_path = "/home/jovyan/MLEAEEE4000-DroughtAmazon2/SAMERICA_WTD_rescaled.tif"

# Read raster
with rasterio.open(raster_path) as src:
    # Convert GeoJSON boundary to GeoJSON-like dict
    geoms = mdd["geometry"].values
    geoms = [geom.__geo_interface__ for geom in geoms]

    # Perform mask (clip)
    out_image, out_transform = mask(src, geoms, crop=True)
    out_meta = src.meta.copy()

In [16]:
# Update metadata with new dimensions, transform, etc.
out_meta.update({
    "driver": "GTiff",
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform
})

with rasterio.open("wtd_clipped_mdd.tif", "w", **out_meta) as dest:
    dest.write(out_image)

Now that we have a resampled WTD at the correct resolution in degrees, and now that we have clipped it to the political boundaries of MDD, Peru... we can mask the WTD raster with the Hansen tree cover extent raster to end up with a raster of WTD for only the areas that have been contiguous forest...

In [17]:
raster_path = "/home/jovyan/MLEAEEE4000-DroughtAmazon2/mdd_extent_TCscale.tif"

with rasterio.open(raster_path) as src:
    data = src.read(1) 

In [18]:
values = data.flatten().tolist()

In [19]:
import numpy as np
unique_values = np.unique(data)

In [20]:
print(unique_values)

[0 1]


In [22]:
rA = "/home/jovyan/MLEAEEE4000-DroughtAmazon2/wtd_clipped_mdd.tif" # the raster to be clipped
rB = "/home/jovyan/MLEAEEE4000-DroughtAmazon2/mdd_extent_TCscale.tif" # the raster to use as a mask
out = "/home/jovyan/MLEAEEE4000-DroughtAmazon2/wtd_masked.tif"

with rasterio.open(rA) as srcA, rasterio.open(rB) as srcB:
    
    A = srcA.read(1)
    B = srcB.read(1)

    # Mask A where B has NoData
    mask = (B == 0)
    A_masked = np.where(mask, srcA.nodata, A)

    # export
    meta = srcA.meta.copy()
    with rasterio.open(out, "w", **meta) as dst:
        dst.write(A_masked, 1)